In [98]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [99]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(train_data.shape, test_data.shape)

(42000, 785) (28000, 784)


In [100]:
labels = train_data['label']
train_data.drop('label', axis=1, inplace=True)

In [101]:
def preprocess(data):
    
    data = data/255
    data -= 0.5
    data = data.values.reshape(data.shape[0],28,28,1)
    return data

In [102]:
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [103]:
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.2, random_state=12)

print(x_train.shape)

(33600, 28, 28, 1)


In [142]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, kernel_size=5, activation='relu', input_shape=(28, 28, 1)))  # 24*24*32
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))                        # 12*12*32
model.add(tf.keras.layers.Conv2D(64, kernel_size=5, activation='relu'))          # 8*8*64
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))                        # 4*4*64

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

adam = tf.keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [108]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=.1, height_shift_range=.1, shear_range=10, zoom_range=[.85, .85])

datagen.fit(x_train)

In [140]:
model.optimizer.lr = .0003
model.fit(x_train, y_train, batch_size=128, epochs=5, shuffle=True)

Epoch 1/5
33600/33600 [==============================] - 59s 2ms/step - loss: 0.0053 - acc: 0.9981
Epoch 2/5
33600/33600 [==============================] - 54s 2ms/step - loss: 0.0056 - acc: 0.9983
Epoch 3/5
33600/33600 [==============================] - 77s 2ms/step - loss: 0.0054 - acc: 0.9982
Epoch 4/5
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0067 - acc: 0.9979
Epoch 5/5
33600/33600 [==============================] - 63s 2ms/step - loss: 0.0065 - acc: 0.9981


In [88]:
epoch = 0
batch = 256
max_epoch = 1
model.optimizer.lr = .0001

for x,y in datagen.flow(x_train, y_train, batch_size=batch):
    
    epoch += 1
    model.fit(x_train, y_train, batch_size=batch)
    if epoch == max_epoch:
        break

Epoch 1/1
33600/33600 [==============================] - 67s 2ms/step - loss: 0.0026 - acc: 0.9993


In [141]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print("\nval loss:", val_loss, "\nval_acc:", val_acc)

8400/8400 [==============================] - 9s 1ms/step

val loss: 0.03671239142301298 
val_acc: 0.991547619047619


In [137]:
predict = model.predict(test_data)
results = predict.argmax(axis=1)
result_data = pd.DataFrame({'ImageId':[x for x in range(1, 28001)], 'Label':results})

result_data.to_csv('my_out.csv', index=False)

In [95]:
predict.shape

(28000, 10)